# FHE Basics


## Introduction
This notebook introduces you to the basic types of operations that you can do with FHE including encoding, decoding, encrypting, decrypting, addition, multiplicaton, and rotation.
<br>  

## Step 1. Import pyhelayers library

Pyhelayers library is the Python API for HELayers. As with all python libraries, before anything can be done, it needs to be imported.

In [ ]:
import pyhelayers
import utils 
utils.verify_memory()
# Print the ciphertext content for demo purposes
pyhelayers.get_print_options().print_encrypted_content=True 
print("Imported pyhelayers", pyhelayers.VERSION)

<br>

## Step 2. Create the context

A `Context` is the main HE object. It receives the configuration parameters, and initializes all the required mathemtical constructs. It also generates the keys and supplies various services. In pyhelayers it also stores the keys.

Configuring the context correctly is key to achieving the desired results when using homomorphic encryption, and its parameters depend on the nature of the problem to be solved.

Pyhelayers library includes the ability for optimizing the parameters of the context, which is explored in the various notebooks. In this notebook, a number of parameters are set manually to illustrate how a `Context` can be configured.

However, before a `Context` can be created, it requires some configuration details to be defined. 

In the example below we create a requirement object and define the security level, integer precision, fractional precision, multiplication depth (how many times the ciphertext can be multiplied), and ciphertext slot size, `num_slots`. A slot defines how large each ciphertext can be. This ciphertext has 8192 slots, i.e., it can hold 8192 numbers.

Once requirements are set, an instance of `Context` is created, called `he_context`, and initialized with the requirements:


In [ ]:

requirement = pyhelayers.HeConfigRequirement(
    num_slots = 8192, # Number of slots per ciphertext
    multiplication_depth = 2, # Allow 2 levels of multiplications
    fractional_part_precision = 40, # Set the precision to 1/2^40.
    integer_part_precision = 20, # Set the largest number to 2^20.
    security_level = 128)

he_context = pyhelayers.DefaultContext()
he_context.init(requirement)

<br>

## Step 3. Perform basic Homomorphic Encryption operations

Once the `Context` has been created and initialized, we can use it for different operations. Next we will showcase basic mathematical operations done using `pyhelayers`. Note that while carrying out these operations the data always remains encrypted and never gets decrypted even during computation.
<br>

### Encoding, decoding, encrypting and decrypting
In HE, the encryption process from the user perspective actually involves two steps: encoding and then encrypting. Similarly the decryption involves decrypting and then decoding.
<br>

The encoding step takes the input data and **encodes** it into plaintext form that can then be **encrypted** into the ciphertext using the public key. The homomorphic computations are done by manipulating the ciphertexts, and when this is done, the holder of the secret key can **decrypt** the resulting ciphertext back into plaintext form. Finally, the plaintext should be **decoded** in order to extract the resulting values (with possible noise).

Thus, for example:

> X + Y =~ decode( decrypt( encrypt( encode(X)) + encrypt(encode(Y)) ) )

For example in some schemes (like CKKS) the input data is a vector of values that need to be encrypted, the encoded plaintext is a polynomial, and the ciphertext is a pair of polynomials.

The role of the encoding step is just to represent the input data in a form that can be encrypted in the encryption step, and it does not provide any security. In other words, anyone may decode the plaintext to reveal the encoded values, without requiring any secret key for this decoding operation. All the security of the scheme comes from the encryption step, which encrypts the encoded plaintext into the ciphertext in such a way that it can only be efficiently decrypted by using the secret key.
<br>
<br>
In this example a ciphertext is created from the vector `[1.0, 2.0, 3.0]`.  Using an instance of `Encoder` named `encoder`, `v1` is encrypted and stored in `c1` by calling `encoder.encode_encrypt(v1)`.  It is then decrypted with `encoder.decrypt_decode_double(c1)` and stored in `d1`.  


The `Encoder` allows converting values to (and from) their encrypted form:

In [ ]:
# Create the Encoder using the context.
encoder = pyhelayers.Encoder(he_context)

# Encode a vector.
v1 = [1.0, 2.0, 3.0]
c1 = encoder.encode_encrypt(v1)
# Decode the encoded vector.
d1 = encoder.decrypt_decode_double(c1)

The previous cell illustrates the "round-trip" encrypting and decrypting of a small vector.  Printing out what is stored in the variables can help illustrate what is happening at each step.  Note that operations in homomorphic encryption involve in a configurable loss of precision:

In [ ]:
print('Initial vector:   {}'.format(v1))
print('Encrypted vector: {}'.format(c1))
print('Decoded vector:   {}'.format(d1[:3]))

The result of encrypting a vector (`c1` in the snippet above) is a `CTile`, which represents a single ciphertext (a single ciphertext in HE usually can hold an entire vector).

The encrypted CTile objects contain a series of methods that can be used for performing operations on them directly, as shown on the following sections.
<br>

### Addition
Adding two encrypted vectors can be performed by the `.add()` method on the first `CTile`:

In [ ]:
v2 = [4.0, 5.0, 6.0]
c2 = encoder.encode_encrypt(v2)
c1.add(c2)
print('Result: {}'.format(encoder.decrypt_decode_double(c1)[:3]))

## Average
Average 3 encrypted vectors can be performed by adding their 3 ciphertexts and multiply with the plaintext of 1/3:

In [ ]:
v1 = [1.0, 5.0, 13.0]
c1 = encoder.encode_encrypt(v1)

v2 = [2.0, 7.0, 17.0]
c2 = encoder.encode_encrypt(v2)

v3 = [3.0, 1.0, 19.0]
c3 = encoder.encode_encrypt(v3)

c1.add(c2) 
c1.add(c3)
c1.multiply_scalar(1/3)

print('Result: {}'.format(encoder.decrypt_decode_double(c1)[:3]))

### Multiplication
In an analog manner to addition, multiplication can be performed using the `.multiply()` method:

In [ ]:
v3 = [7.0, 8.0, 9.0]
c3 = encoder.encode_encrypt(v3)
c1.multiply(c3)
print('Result: {}'.format(encoder.decrypt_decode_double(c1)[:3]))

### Rotation

As another example of basic operations, the elements of an encrypted vector can be rotated by calling the `.rotate()` method:

In [ ]:
c1.rotate(-1)
print('Result: {}'.format(encoder.decrypt_decode_double(c1)[:3]))

<br>

## Step 4. Further help

The pyhelayers documentation can be viewed using the 'help' function.  
For example: `help(pyhelayers.CTile)`

<br>

## Step 5. Up next

 The other notebooks in this tutorial showcase how these primitives can be used in more realistic scenarios and expand on other concepts in the pyhelayers library. Try out the Neural Network Fraud Detection notebook next to understand how FHE can be applied to the Financial Services industry.